<a href="https://colab.research.google.com/github/bharti0420/Air-Pollution-Detection-Prediction/blob/master/StationaryOnPM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install arch

In [0]:
from pandas import DataFrame
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from datetime import *
import csv
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib.pylab import rcParams
%matplotlib inline
rcParams['figure.figsize']=10,6

In [0]:
from arch.unitroot import ADF

In [0]:
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import coint, adfuller, kpss

In [0]:
import operator

In [0]:
def testStationaryAdfuller(file_name,year,ts,cutoff=0.05):
  s="Result of "+file_name+" site and year" + year
  print(s)
  ts_test=ADF(ts)
  print ('Results of ADF Test:')
  print(ts_test)
  p_value=ts_test.pvalue
  p_value=round(p_value,5)
  if p_value <= cutoff:
    print("\nStrong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary\n")
  else:
    print("\nWeak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary\n")

In [0]:
def dataFilter(file_name, lengthList):
  df=pd.read_csv(file_name)
  #print(len(df))
  lengthList[file_name]=len(df)
  df['from date'] = pd.to_datetime(df['from date'])
  df['to date'] = pd.to_datetime(df['to date'])
  
  #set index (from date is index)
  df.set_index(['from date'],drop=True, inplace=True)
  
  df = df[pd.notnull(df['PM2.5'])]
  #print(len(df))
  lengthList[file_name]=(len(df)/lengthList[file_name])*100
  years = [g for n, g in df.set_index('to date').groupby(pd.TimeGrouper('A'))]
  print("Total Year:\n",len(years))
  year_list=['2016','2017','2018']
  for i in range(0,3):
    df=years[i]
    testStationaryAdfuller(file_name,year_list[i],df['PM2.5'])
  
  

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
lengthList=dict()
for fn in uploaded.keys():
  dataFilter(fn,lengthList)
  print("-----------------------------------------------------------------------------------------------------------\n\n")
print("\n\n")
print(lengthList)
baseStation=max(lengthList.items(), key=operator.itemgetter(1))[0]
print("Base Station: ",baseStation)

### Granger Causality test

In [0]:
#from google.colab import files
#uploaded = files.upload()

Saving complete146.csv to complete146 (1).csv
Saving complete147.csv to complete147.csv


In [0]:
filename_list=[]
for fn in uploaded.keys():
  filename_list.append(fn)
#print(filename_list)

In [0]:
df_146=pd.read_csv(filename_list[0])
df_147=pd.read_csv(filename_list[1])

In [0]:
print(len(df_146))
print(len(df_147))

32779
105120


In [0]:
df_146['from date']=pd.to_datetime(df_146['from date'])
df_146['to date']=pd.to_datetime(df_146['to date'])

df_147['from date']=pd.to_datetime(df_147['from date'])
df_147['to date']=pd.to_datetime(df_147['to date'])

In [0]:
df_146=df_146[pd.notnull(df_146['PM2.5'])]
df_147=df_147[pd.notnull(df_147['PM2.5'])]


In [0]:
df_146.rename(columns={'PM2.5':'PM2.5_146'},inplace=True)
df_147.rename(columns={'PM2.5':'PM2.5_147'},inplace=True)


In [0]:
df=pd.merge(df_146,df_147,how='inner',on=['from date','to date'])
list1=list(df['from date'])
list2=list(df['to date'])
pm_146=list(df['PM2.5_146'])
pm_147=list(df['PM2.5_147'])
print(len(list1))
col_list=['from date','to date','PM2.5_146','PM2.5_147']
df=df[col_list]
from statsmodels.tsa.stattools  import   grangercausalitytests
print(grangercausalitytests(df[['PM2.5_146', 'PM2.5_147']], maxlag=5, addconst=True, verbose=True))




16216


In [0]:
def granger_causality_test(df_delhi_124,df_common,site_name):
  df_delhi_124['from date']=pd.to_datetime(df_delhi_124['from date'])
  df_delhi_124['to date']=pd.to_datetime(df_delhi_124['to date'])
  df_common['from date']=pd.to_datetime(df_common['from date'])
  df_common['to date']=pd.to_datetime(df_common['to date'])
  
  #remove null value of PM2.5
  df_common=df_common[pd.notnull(df_common['PM2.5'])]
  
  #intersection of both dataframe for PM2.5 of same time
  df=pd.merge(df_delhi_124,df_common,how='inner',on=['from date','to date'])
  col_list=['from date','to date','PM2.5_124','PM2.5']
  df=df[col_list]
  
  print("Granger Test between Delhi site 124 and the "+ site_name+".")
  
  print(grangercausalitytests(df[['PM2.5_124', 'PM2.5']], maxlag=5, addconst=True, verbose=True))
  print("-------------------------------------------------------------------------------------------------------------\n\n")
  

### For Delhi Site

In [3]:
from google.colab import files
uploaded = files.upload()

Saving 124_complete.csv to 124_complete.csv


In [4]:
delhi_site1=""
for fn in uploaded.keys():
  delhi_site1=fn
delhi_site=delhi_site1

print(delhi_site)

124_complete.csv


In [0]:
df_delhi_124=pd.read_csv(delhi_site)
df_delhi_124=df_delhi_124[pd.notnull(df_delhi_124['PM2.5'])]
df_delhi_124.rename(columns={'PM2.5':'PM2.5_124'},inplace=True)


### For Haryana sites

In [16]:
from google.colab import files
uploaded = files.upload()

Saving complete146.csv to complete146.csv
Saving complete147.csv to complete147.csv
Saving complete161.csv to complete161.csv
Saving complete263.csv to complete263.csv


In [17]:
haryana_list=[]
for fn in uploaded.keys():
  haryana_list.append(fn)
print(haryana_list)

  

['complete146.csv', 'complete147.csv', 'complete161.csv', 'complete263.csv']


In [20]:
site_list=['146','147','161','163']
for i in range(len(haryana_list)):
  df=pd.read_csv(haryana_list[i])
  site_name='Haryana site '+site_list[i]
  granger_causality_test(df_delhi_124,df,site_name)
  print("\n")
#print("-------------------------------------------------------------------------------------------------------------\n\n")

Granger Test between Delhi site 124 and the Haryana site 146.

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=63.1059 , p=0.0000  , df_denom=75773, df_num=1
ssr based chi2 test:   chi2=63.1084 , p=0.0000  , df=1
likelihood ratio test: chi2=63.0822 , p=0.0000  , df=1
parameter F test:         F=63.1059 , p=0.0000  , df_denom=75773, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=34.3645 , p=0.0000  , df_denom=75770, df_num=2
ssr based chi2 test:   chi2=68.7335 , p=0.0000  , df=2
likelihood ratio test: chi2=68.7023 , p=0.0000  , df=2
parameter F test:         F=34.3645 , p=0.0000  , df_denom=75770, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=26.3545 , p=0.0000  , df_denom=75767, df_num=3
ssr based chi2 test:   chi2=79.0707 , p=0.0000  , df=3
likelihood ratio test: chi2=79.0295 , p=0.0000  , df=3
parameter F test:         F=26.3545 , p=0.0000  , df_denom=75767, df_num=3

Granger Causality
n

### For Panjab Site

In [22]:
from google.colab import files
uploaded = files.upload()

Saving complete253.csv to complete253.csv
Saving complete256.csv to complete256.csv
Saving complete1437.csv to complete1437.csv
Saving complete1438.csv to complete1438.csv
Saving complete1450.csv to complete1450.csv
Saving complete1538.csv to complete1538.csv
Saving complete1549.csv to complete1549.csv


In [23]:
panjab_list=[]
for fn in uploaded.keys():
  panjab_list.append(fn)
print(panjab_list)

['complete253.csv', 'complete256.csv', 'complete1437.csv', 'complete1438.csv', 'complete1450.csv', 'complete1538.csv', 'complete1549.csv']


In [25]:
site_list=['253','256','1437','1438','1450','1538','1549']
for i in range(len(panjab_list)):
  df=pd.read_csv(panjab_list[i])
  site_name='Panjab site '+site_list[i]
  granger_causality_test(df_delhi_124,df,site_name)
  print("\n")
  

Granger Test between Delhi site 124 and the Panjab site 253.

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.6074 , p=0.0000  , df_denom=47109, df_num=1
ssr based chi2 test:   chi2=46.6103 , p=0.0000  , df=1
likelihood ratio test: chi2=46.5873 , p=0.0000  , df=1
parameter F test:         F=46.6074 , p=0.0000  , df_denom=47109, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=30.2140 , p=0.0000  , df_denom=47106, df_num=2
ssr based chi2 test:   chi2=60.4344 , p=0.0000  , df=2
likelihood ratio test: chi2=60.3956 , p=0.0000  , df=2
parameter F test:         F=30.2140 , p=0.0000  , df_denom=47106, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=20.1033 , p=0.0000  , df_denom=47103, df_num=3
ssr based chi2 test:   chi2=60.3190 , p=0.0000  , df=3
likelihood ratio test: chi2=60.2804 , p=0.0000  , df=3
parameter F test:         F=20.1033 , p=0.0000  , df_denom=47103, df_num=3

Granger Causality
nu

### For Rajsthan Site

In [26]:
from google.colab import files
uploaded = files.upload()

Saving complete134.csv to complete134.csv
Saving complete1390.csv to complete1390.csv
Saving complete1392.csv to complete1392.csv
Saving complete1393.csv to complete1393.csv
Saving complete1394.csv to complete1394.csv
Saving complete1395.csv to complete1395.csv
Saving complete1396.csv to complete1396.csv
Saving complete1397.csv to complete1397.csv


In [27]:
Rajsthan_list=[]
for fn in uploaded.keys():
  Rajsthan_list.append(fn)
print(Rajsthan_list)

['complete134.csv', 'complete1390.csv', 'complete1392.csv', 'complete1393.csv', 'complete1394.csv', 'complete1395.csv', 'complete1396.csv', 'complete1397.csv']


In [0]:
site_list=['134','1390','1392','1393','1394','1395','1396','1397']
for i in range(len(Rajsthan_list)):
  df=pd.read_csv(Rajsthan_list[i])
  site_name='Rajsthan site '+site_list[i]
  granger_causality_test(df_delhi_124,df,site_name)
  print("\n")
  print("-------------------------------------------------------------------------------------------------------------\n\n")

Granger Test between Delhi site 124 and the Rajsthan site 134.

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=41.6402 , p=0.0000  , df_denom=50861, df_num=1
ssr based chi2 test:   chi2=41.6427 , p=0.0000  , df=1
likelihood ratio test: chi2=41.6257 , p=0.0000  , df=1
parameter F test:         F=41.6402 , p=0.0000  , df_denom=50861, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=23.8093 , p=0.0000  , df_denom=50858, df_num=2
ssr based chi2 test:   chi2=47.6233 , p=0.0000  , df=2
likelihood ratio test: chi2=47.6010 , p=0.0000  , df=2
parameter F test:         F=23.8093 , p=0.0000  , df_denom=50858, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=19.1402 , p=0.0000  , df_denom=50855, df_num=3
ssr based chi2 test:   chi2=57.4286 , p=0.0000  , df=3
likelihood ratio test: chi2=57.3962 , p=0.0000  , df=3
parameter F test:         F=19.1402 , p=0.0000  , df_denom=50855, df_num=3

Granger Causality


### For UP Site

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
Up_list=[]
for fn in uploaded.keys():
  Up_list.append(fn)
print(Up_list)

In [0]:
site_list=['144','153','272','277','297','307','1398','1405','1429','1541','1542','1550','1569']
for i in range(len(Up_list)):
  df=pd.read_csv(Up_list[i])
  site_name='Up site '+site_list[i]
  granger_causality_test(df_delhi_124,df,site_name)
  print("\n")
  print("-------------------------------------------------------------------------------------------------------------\n\n")